In [1]:
import os
import json

In [2]:
# the pipeline
# def extract_repo_id_code(row):
#     if 2012 <= row['year'] <= 2014:
#         return row['repository']['id']
#     else:
#         return row['repo']['id']
# 
# def fork_event_pipeline(ctx, input_pattern, s3_output_path):
#     """test pipeline to extract fork evenets across years"""
#     ctx.json(input_pattern) \
#         .withColumn('year', lambda x: int(x['created_at'].split('-')[0])) \
#         .withColumn('repo_id', extract_repo_id_code) \
#         .filter(lambda x: x['type'] == 'ForkEvent') \
#         .selectColumns(['type', 'repo_id', 'year']) \
#         .tocsv(s3_output_path)
# 

In [3]:
def extract_repo_id_code(row):
    if 2012 <= row['year'] <= 2014:
        return row['repository']['id']
    else:
        return row['repo']['id']

In [4]:
 from tqdm import tqdm

In [14]:
def process_row(row_no, line, op):
    row = json.loads(line.strip())
    row['year'] = int(row['created_at'].split('-')[0])
    row['repo_id'] = extract_repo_id_code(row)
    if row['type'] == 'ForkEvent':
        op.write('{}\n'.format(','.join([str(row['type']), str(row['repo_id']), str(row['year']), str(row_no)])))

In [15]:
input_path = '/home/leonhard/Downloads/2011-10-15.json'
output_path = '/home/leonhard/Downloads/2011-10-15.csv'

exceptions = []

num_normal_rows = 0
with open(input_path, 'r') as fp:
    with open(output_path, 'w') as op:
        for row_no, line in tqdm(enumerate(fp.readlines())):
            try:
                process_row(row_no, line, op)
                num_normal_rows += 1
            except Exception as e:
                exceptions.append((row_no, line, str(e), str(type(e))))
print('processed in total {} normal rows, {} exceptions'.format(num_normal_rows, len(exceptions)))

48899it [00:00, 90249.92it/s]

processed in total 47615 normal rows, 1284 exceptions


In [19]:
set(list(map(lambda t: t[3], exceptions)))

{"<class 'KeyError'>"}

In [20]:
# -> 2011 has 1284 KeyError exceptions.

In [33]:
input_path = '/home/leonhard/Downloads/2013-10-15.json'
output_path = '/home/leonhard/Downloads/2013-10-15.csv'

exceptions = []

num_normal_rows = 0
with open(input_path, 'r') as fp:
    with open(output_path, 'w') as op:
        for row_no, line in tqdm(enumerate(fp.readlines())):
            try:
                process_row(row_no, line, op)
                num_normal_rows += 1
            except Exception as e:
                exceptions.append((row_no, line, str(e), str(type(e))))
print('processed in total {} normal rows, {} exceptions'.format(num_normal_rows, len(exceptions)))

296456it [00:04, 72303.31it/s]

processed in total 285731 normal rows, 10725 exceptions


In [34]:
set(list(map(lambda t: t[3], exceptions)))

{"<class 'KeyError'>"}

In [35]:
# 2013 has 10725 true exceptions.

In [37]:
exceptions[0][1]

'{"created_at":"2013-10-15T00:02:19-07:00","payload":{"id":6987642,"url":"https://gist.github.com/6987642","name":"gist: 6987642","desc":"Bootstrap Customizer Config","action":"create"},"public":true,"type":"GistEvent","url":"https://gist.github.com/6987642","actor":null}\n'

In [38]:
# are there any fork events?

In [39]:
[event[1] for event in exceptions if 'ForkEvent' in event[1]]

[]

In [41]:
set([json.loads(event[1])['type'] for event in exceptions])

{'CreateEvent', 'FollowEvent', 'GistEvent', 'MemberEvent', 'PushEvent'}

In [25]:
# with filter pushdown, this would change.

In [32]:
124065 + 172391

296456

In [26]:
# but for now, NO filter pushdown.

2011:
    
first exception buffer has 10789 rows:

=> split into
10789 = 285 (exceptions) + 1039 (general) + 9465 (interpreter)

11661 = 329 (exceptions) + 1382 (general) + 9950 (interpreter)

11602 = 351 (exceptions) + 1504 (general) + 9747 (interpreter)

11321 = 319 (exceptions) + 1188 (general) + 9814 (interpreter)



In [27]:
285 + 329 + 351 + 319

1284

2013:

reports in total 285745 normal rows (good rows) and 10711 exception rows.

--> some except rows are counted as good rows (which shouldn't have been counted as such.)

however, should be: 285731 normal rows, 10725 exceptions


Test with interpreter only. (is it a counting error from general path?)

In [42]:
!wc -l '/home/leonhard/Downloads/2013-10-15.csv'


10628 /home/leonhard/Downloads/2013-10-15.csv


In [43]:
import json
import csv
import io
import cloudpickle

# helper row object to allow fancy integer and column based string access within UDFs!
class Row:
    def __init__(self, data, columns=None):
        assert(isinstance(data, (tuple, list)))
        assert(isinstance(columns, (tuple, list)) or columns is None)
        self.data = tuple(data)
        self.columns = tuple(columns[:len(data)]) if columns is not None else None 
        
    def __getitem__(self, key):
        # check for int also works for bool!
        if isinstance(key, int):
            return self.data[key]
        # getitem either gets a key or slice object
        elif isinstance(key, slice):
                   return self.data[key.start:key.stop:key.step]
        elif isinstance(key, str):
            if self.columns is None:
                raise KeyError("no columns defined, can't access column '{}'".format(key))
            elif key not in self.columns:
                raise KeyError("could not find column column '{}'".format(key))
            return self.data[self.columns.index(key)]
        else:
            raise IndexError()
            
    def __repr__(self):
        if self.columns:
            if len(self.columns) < len(self.data):
                self.columns = self.columns + [None] * (len(self.data) - len(self.columns))
            return '(' + ','.join(['{}={}'.format(c, d) for c, d in zip(self.columns, self.data)]) + ')'
        else:
            return '(' + ','.join(['{}'.format(d) for d in self.data]) + ')'
# recursive expansion of Row objects potentially present in data.
def expand_row(x):
    # Note: need to use here type construction, because isinstance fails for dict input when checking for list
    if hasattr(type(x), '__iter__') and not isinstance(x, str):
        if type(x) is tuple:
            return tuple([expand_row(el) for el in x])
        elif type(x) is list:
            return [expand_row(el) for el in x]
        elif type(x) is dict:
            return {expand_row(key) : expand_row(val) for key, val in x.items()}
        else:
            raise TypeError("custom sequence type used, can't convert to data representation")
    return x.data if isinstance(x, Row) else x

def result_to_row(res, columns=None):
    # convert result to row object, i.e. deal with unpacking etc.
    # is result a dict?
    if type(res) is dict:
        # are all keys strings? If so, then unpack!
        # else, keep it as dict return object!
        if all(map(lambda k: type(k) == str, res.keys())):
            # columns become keys, values 
            columns = tuple(res.keys())
            data = tuple(map(lambda k: res[k], columns))
            return Row(data, columns)
    
    
    # is it a row object?
    # => convert to tuple!
    r = expand_row(res)
    
    if type(r) is not tuple:
        r = (r,)
    else:
        if len(r) == 0:
            r = ((),) # special case, empty tuple
    
    return Row(r, columns)

def apply_func(f, row):
    if len(row.data) != 1:
        # check how many positional arguments function has.
        # if not one, expand row into multi args!
        nargs = f.__code__.co_argcount
        if nargs != 1:
            return f(*tuple([row[i] for i in range(nargs)]))
        else:
            return f(row)
    else:
        # unwrap single element tuples.
        return f(row.data[0])

def pipeline_stage_0(input_row, parse_cells=True):
    res = {'outputRows':[]}
    for _ in range(1):
        if parse_cells:
            if not isinstance(input_row[0], str):
                res['exception'] = TypeError('json input must be of string type')
                res['exceptionOperatorID'] = 100000
                res['inputRow'] = input_row
                return res
            parsed_row = json.loads(input_row[0])
            keys = ['type','public','actor','created_at','payload','id','repo','org',]
            tmp_row = parsed_row
            left_over_keys = list(set(tmp_row.keys()) - set(keys))
            parsed_row = [tmp_row.get(k, None) for k in keys] + [tmp_row.get(k, None) for k in left_over_keys]
            row = Row(parsed_row, keys + left_over_keys)
            res['outputColumns'] = keys + left_over_keys
        else:
            parsed_row = input_row
            row = Row(parsed_row, ['type','public','actor','created_at','payload','id','repo','org',])
            res['outputColumns'] = ['type','public','actor','created_at','payload','id','repo','org',]
        try:
            code = b'\x80\x05\x95\xdb\x01\x00\x00\x00\x00\x00\x00\x8c\x17\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x2e\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x94\x8c\x0e\x5f\x6d\x61\x6b\x65\x5f\x66\x75\x6e\x63\x74\x69\x6f\x6e\x94\x93\x94\x28\x68\x00\x8c\x0d\x5f\x62\x75\x69\x6c\x74\x69\x6e\x5f\x74\x79\x70\x65\x94\x93\x94\x8c\x08\x43\x6f\x64\x65\x54\x79\x70\x65\x94\x85\x94\x52\x94\x28\x4b\x01\x4b\x00\x4b\x00\x4b\x01\x4b\x04\x4b\x43\x43\x16\x74\x00\x7c\x00\x64\x01\x19\x00\xa0\x01\x64\x02\xa1\x01\x64\x03\x19\x00\x83\x01\x53\x00\x94\x28\x4e\x8c\x0a\x63\x72\x65\x61\x74\x65\x64\x5f\x61\x74\x94\x8c\x01\x2d\x94\x4b\x00\x74\x94\x8c\x03\x69\x6e\x74\x94\x8c\x05\x73\x70\x6c\x69\x74\x94\x86\x94\x8c\x01\x78\x94\x85\x94\x8c\x08\x3c\x73\x74\x72\x69\x6e\x67\x3e\x94\x8c\x08\x3c\x6c\x61\x6d\x62\x64\x61\x3e\x94\x4b\x01\x43\x02\x16\x00\x94\x29\x29\x74\x94\x52\x94\x7d\x94\x28\x8c\x0b\x5f\x5f\x70\x61\x63\x6b\x61\x67\x65\x5f\x5f\x94\x4e\x8c\x08\x5f\x5f\x6e\x61\x6d\x65\x5f\x5f\x94\x8c\x08\x5f\x5f\x6d\x61\x69\x6e\x5f\x5f\x94\x75\x4e\x4e\x4e\x74\x94\x52\x94\x8c\x1c\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x2e\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x5f\x66\x61\x73\x74\x94\x8c\x12\x5f\x66\x75\x6e\x63\x74\x69\x6f\x6e\x5f\x73\x65\x74\x73\x74\x61\x74\x65\x94\x93\x94\x68\x1b\x7d\x94\x7d\x94\x28\x68\x18\x68\x12\x8c\x0c\x5f\x5f\x71\x75\x61\x6c\x6e\x61\x6d\x65\x5f\x5f\x94\x68\x12\x8c\x0f\x5f\x5f\x61\x6e\x6e\x6f\x74\x61\x74\x69\x6f\x6e\x73\x5f\x5f\x94\x7d\x94\x8c\x0e\x5f\x5f\x6b\x77\x64\x65\x66\x61\x75\x6c\x74\x73\x5f\x5f\x94\x4e\x8c\x0c\x5f\x5f\x64\x65\x66\x61\x75\x6c\x74\x73\x5f\x5f\x94\x4e\x8c\x0a\x5f\x5f\x6d\x6f\x64\x75\x6c\x65\x5f\x5f\x94\x68\x19\x8c\x07\x5f\x5f\x64\x6f\x63\x5f\x5f\x94\x4e\x8c\x0b\x5f\x5f\x63\x6c\x6f\x73\x75\x72\x65\x5f\x5f\x94\x4e\x8c\x17\x5f\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x5f\x73\x75\x62\x6d\x6f\x64\x75\x6c\x65\x73\x94\x5d\x94\x8c\x0b\x5f\x5f\x67\x6c\x6f\x62\x61\x6c\x73\x5f\x5f\x94\x7d\x94\x75\x86\x94\x86\x52\x30\x2e'
            f = cloudpickle.loads(code)
            call_res = apply_func(f, row)
            if row.columns and 'year' in row.columns:
                col_idx = row.columns.index('year')
                tmp = list(row.data)
                tmp[col_idx] = expand_row(call_res)
                row.data = tuple(tmp)
            else:
                row.columns = row.columns + ('year',) if row.columns is not None else tuple([None] * len(row.data)) + ('year',)
                row.data = row.data + result_to_row(call_res).data
        except Exception as e:
            res['exception'] = e
            res['exceptionOperatorID'] = 100001
            res['inputRow'] = input_row
        try:
            code = b'\x80\x05\x95\x0a\x02\x00\x00\x00\x00\x00\x00\x8c\x17\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x2e\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x94\x8c\x0e\x5f\x6d\x61\x6b\x65\x5f\x66\x75\x6e\x63\x74\x69\x6f\x6e\x94\x93\x94\x28\x68\x00\x8c\x0d\x5f\x62\x75\x69\x6c\x74\x69\x6e\x5f\x74\x79\x70\x65\x94\x93\x94\x8c\x08\x43\x6f\x64\x65\x54\x79\x70\x65\x94\x85\x94\x52\x94\x28\x4b\x01\x4b\x00\x4b\x00\x4b\x01\x4b\x03\x4b\x43\x43\x34\x64\x01\x7c\x00\x64\x02\x19\x00\x04\x00\x03\x00\x6b\x01\x72\x0c\x64\x03\x6b\x01\x72\x14\x6e\x02\x01\x00\x6e\x06\x7c\x00\x64\x04\x19\x00\x64\x05\x19\x00\x53\x00\x7c\x00\x64\x06\x19\x00\x64\x05\x19\x00\x53\x00\x94\x28\x4e\x4d\xdc\x07\x8c\x04\x79\x65\x61\x72\x94\x4d\xde\x07\x8c\x0a\x72\x65\x70\x6f\x73\x69\x74\x6f\x72\x79\x94\x8c\x02\x69\x64\x94\x8c\x04\x72\x65\x70\x6f\x94\x74\x94\x29\x8c\x03\x72\x6f\x77\x94\x85\x94\x8c\x08\x3c\x73\x74\x72\x69\x6e\x67\x3e\x94\x8c\x0f\x65\x78\x74\x72\x61\x63\x74\x5f\x72\x65\x70\x6f\x5f\x69\x64\x94\x4b\x01\x43\x06\x1c\x01\x0c\x01\x0c\x02\x94\x29\x29\x74\x94\x52\x94\x7d\x94\x28\x8c\x0b\x5f\x5f\x70\x61\x63\x6b\x61\x67\x65\x5f\x5f\x94\x4e\x8c\x08\x5f\x5f\x6e\x61\x6d\x65\x5f\x5f\x94\x8c\x08\x5f\x5f\x6d\x61\x69\x6e\x5f\x5f\x94\x75\x4e\x4e\x4e\x74\x94\x52\x94\x8c\x1c\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x2e\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x5f\x66\x61\x73\x74\x94\x8c\x12\x5f\x66\x75\x6e\x63\x74\x69\x6f\x6e\x5f\x73\x65\x74\x73\x74\x61\x74\x65\x94\x93\x94\x68\x1a\x7d\x94\x7d\x94\x28\x68\x17\x68\x11\x8c\x0c\x5f\x5f\x71\x75\x61\x6c\x6e\x61\x6d\x65\x5f\x5f\x94\x68\x11\x8c\x0f\x5f\x5f\x61\x6e\x6e\x6f\x74\x61\x74\x69\x6f\x6e\x73\x5f\x5f\x94\x7d\x94\x8c\x0e\x5f\x5f\x6b\x77\x64\x65\x66\x61\x75\x6c\x74\x73\x5f\x5f\x94\x4e\x8c\x0c\x5f\x5f\x64\x65\x66\x61\x75\x6c\x74\x73\x5f\x5f\x94\x4e\x8c\x0a\x5f\x5f\x6d\x6f\x64\x75\x6c\x65\x5f\x5f\x94\x68\x18\x8c\x07\x5f\x5f\x64\x6f\x63\x5f\x5f\x94\x4e\x8c\x0b\x5f\x5f\x63\x6c\x6f\x73\x75\x72\x65\x5f\x5f\x94\x4e\x8c\x17\x5f\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x5f\x73\x75\x62\x6d\x6f\x64\x75\x6c\x65\x73\x94\x5d\x94\x8c\x0b\x5f\x5f\x67\x6c\x6f\x62\x61\x6c\x73\x5f\x5f\x94\x7d\x94\x75\x86\x94\x86\x52\x30\x2e'
            f = cloudpickle.loads(code)
            call_res = apply_func(f, row)
            if row.columns and 'repo_id' in row.columns:
                col_idx = row.columns.index('repo_id')
                tmp = list(row.data)
                tmp[col_idx] = expand_row(call_res)
                row.data = tuple(tmp)
            else:
                row.columns = row.columns + ('repo_id',) if row.columns is not None else tuple([None] * len(row.data)) + ('repo_id',)
                row.data = row.data + result_to_row(call_res).data
        except Exception as e:
            res['exception'] = e
            res['exceptionOperatorID'] = 100002
            res['inputRow'] = input_row
        try:
            code = b'\x80\x05\x95\xc1\x01\x00\x00\x00\x00\x00\x00\x8c\x17\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x2e\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x94\x8c\x0e\x5f\x6d\x61\x6b\x65\x5f\x66\x75\x6e\x63\x74\x69\x6f\x6e\x94\x93\x94\x28\x68\x00\x8c\x0d\x5f\x62\x75\x69\x6c\x74\x69\x6e\x5f\x74\x79\x70\x65\x94\x93\x94\x8c\x08\x43\x6f\x64\x65\x54\x79\x70\x65\x94\x85\x94\x52\x94\x28\x4b\x01\x4b\x00\x4b\x00\x4b\x01\x4b\x02\x4b\x43\x43\x0c\x7c\x00\x64\x01\x19\x00\x64\x02\x6b\x02\x53\x00\x94\x4e\x8c\x04\x74\x79\x70\x65\x94\x8c\x09\x46\x6f\x72\x6b\x45\x76\x65\x6e\x74\x94\x87\x94\x29\x8c\x01\x78\x94\x85\x94\x8c\x08\x3c\x73\x74\x72\x69\x6e\x67\x3e\x94\x8c\x08\x3c\x6c\x61\x6d\x62\x64\x61\x3e\x94\x4b\x01\x43\x02\x0c\x00\x94\x29\x29\x74\x94\x52\x94\x7d\x94\x28\x8c\x0b\x5f\x5f\x70\x61\x63\x6b\x61\x67\x65\x5f\x5f\x94\x4e\x8c\x08\x5f\x5f\x6e\x61\x6d\x65\x5f\x5f\x94\x8c\x08\x5f\x5f\x6d\x61\x69\x6e\x5f\x5f\x94\x75\x4e\x4e\x4e\x74\x94\x52\x94\x8c\x1c\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x2e\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x5f\x66\x61\x73\x74\x94\x8c\x12\x5f\x66\x75\x6e\x63\x74\x69\x6f\x6e\x5f\x73\x65\x74\x73\x74\x61\x74\x65\x94\x93\x94\x68\x18\x7d\x94\x7d\x94\x28\x68\x15\x68\x0f\x8c\x0c\x5f\x5f\x71\x75\x61\x6c\x6e\x61\x6d\x65\x5f\x5f\x94\x68\x0f\x8c\x0f\x5f\x5f\x61\x6e\x6e\x6f\x74\x61\x74\x69\x6f\x6e\x73\x5f\x5f\x94\x7d\x94\x8c\x0e\x5f\x5f\x6b\x77\x64\x65\x66\x61\x75\x6c\x74\x73\x5f\x5f\x94\x4e\x8c\x0c\x5f\x5f\x64\x65\x66\x61\x75\x6c\x74\x73\x5f\x5f\x94\x4e\x8c\x0a\x5f\x5f\x6d\x6f\x64\x75\x6c\x65\x5f\x5f\x94\x68\x16\x8c\x07\x5f\x5f\x64\x6f\x63\x5f\x5f\x94\x4e\x8c\x0b\x5f\x5f\x63\x6c\x6f\x73\x75\x72\x65\x5f\x5f\x94\x4e\x8c\x17\x5f\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x5f\x73\x75\x62\x6d\x6f\x64\x75\x6c\x65\x73\x94\x5d\x94\x8c\x0b\x5f\x5f\x67\x6c\x6f\x62\x61\x6c\x73\x5f\x5f\x94\x7d\x94\x75\x86\x94\x86\x52\x30\x2e'
            f = cloudpickle.loads(code)
            call_res = apply_func(f, row)
            if not call_res:
                continue
        except Exception as e:
            res['exception'] = e
            res['exceptionOperatorID'] = 100003
            res['inputRow'] = input_row
        try:
            code = b'\x80\x05\x95\xd1\x01\x00\x00\x00\x00\x00\x00\x8c\x17\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x2e\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x94\x8c\x0e\x5f\x6d\x61\x6b\x65\x5f\x66\x75\x6e\x63\x74\x69\x6f\x6e\x94\x93\x94\x28\x68\x00\x8c\x0d\x5f\x62\x75\x69\x6c\x74\x69\x6e\x5f\x74\x79\x70\x65\x94\x93\x94\x8c\x08\x43\x6f\x64\x65\x54\x79\x70\x65\x94\x85\x94\x52\x94\x28\x4b\x01\x4b\x00\x4b\x00\x4b\x01\x4b\x04\x4b\x43\x43\x16\x7c\x00\x64\x01\x19\x00\x7c\x00\x64\x02\x19\x00\x7c\x00\x64\x03\x19\x00\x66\x03\x53\x00\x94\x28\x4e\x8c\x04\x74\x79\x70\x65\x94\x8c\x07\x72\x65\x70\x6f\x5f\x69\x64\x94\x8c\x04\x79\x65\x61\x72\x94\x74\x94\x29\x8c\x01\x74\x94\x85\x94\x8c\x08\x3c\x73\x74\x72\x69\x6e\x67\x3e\x94\x8c\x08\x3c\x6c\x61\x6d\x62\x64\x61\x3e\x94\x4b\x01\x43\x02\x16\x00\x94\x29\x29\x74\x94\x52\x94\x7d\x94\x28\x8c\x0b\x5f\x5f\x70\x61\x63\x6b\x61\x67\x65\x5f\x5f\x94\x4e\x8c\x08\x5f\x5f\x6e\x61\x6d\x65\x5f\x5f\x94\x8c\x08\x5f\x5f\x6d\x61\x69\x6e\x5f\x5f\x94\x75\x4e\x4e\x4e\x74\x94\x52\x94\x8c\x1c\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x2e\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x5f\x66\x61\x73\x74\x94\x8c\x12\x5f\x66\x75\x6e\x63\x74\x69\x6f\x6e\x5f\x73\x65\x74\x73\x74\x61\x74\x65\x94\x93\x94\x68\x19\x7d\x94\x7d\x94\x28\x68\x16\x68\x10\x8c\x0c\x5f\x5f\x71\x75\x61\x6c\x6e\x61\x6d\x65\x5f\x5f\x94\x68\x10\x8c\x0f\x5f\x5f\x61\x6e\x6e\x6f\x74\x61\x74\x69\x6f\x6e\x73\x5f\x5f\x94\x7d\x94\x8c\x0e\x5f\x5f\x6b\x77\x64\x65\x66\x61\x75\x6c\x74\x73\x5f\x5f\x94\x4e\x8c\x0c\x5f\x5f\x64\x65\x66\x61\x75\x6c\x74\x73\x5f\x5f\x94\x4e\x8c\x0a\x5f\x5f\x6d\x6f\x64\x75\x6c\x65\x5f\x5f\x94\x68\x17\x8c\x07\x5f\x5f\x64\x6f\x63\x5f\x5f\x94\x4e\x8c\x0b\x5f\x5f\x63\x6c\x6f\x73\x75\x72\x65\x5f\x5f\x94\x4e\x8c\x17\x5f\x63\x6c\x6f\x75\x64\x70\x69\x63\x6b\x6c\x65\x5f\x73\x75\x62\x6d\x6f\x64\x75\x6c\x65\x73\x94\x5d\x94\x8c\x0b\x5f\x5f\x67\x6c\x6f\x62\x61\x6c\x73\x5f\x5f\x94\x7d\x94\x75\x86\x94\x86\x52\x30\x2e'
            f = cloudpickle.loads(code)
            call_res = apply_func(f, row)
            row = result_to_row(call_res, row.columns)
        except Exception as e:
            res['exception'] = e
            res['exceptionOperatorID'] = 100004
            res['inputRow'] = input_row
        buf = io.StringIO()
        w = csv.writer(buf, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        w.writerow(row.data)
        csvSerialized = buf.getvalue()
        res['outputRows'] += [csvSerialized]
        res['outputColumns'] = row.columns
    return res



In [46]:
input_path = '/home/leonhard/Downloads/2013-10-15.json'
output_path = '/home/leonhard/Downloads/2013-10-15.csv'

exceptions = []

num_normal_rows = 0
with open(input_path, 'r') as fp:
    with open(output_path, 'w') as op:
        for row_no, line in tqdm(enumerate(fp.readlines())):
            res = pipeline_stage_0([line])
            if 'exception' in res.keys():
                exceptions.append(res)
            else:
            #process_row(row_no, line, op)
                num_normal_rows += 1
            
print('processed in total {} normal rows, {} exceptions'.format(num_normal_rows, len(exceptions)))

296456it [00:19, 15245.71it/s]

processed in total 285731 normal rows, 10725 exceptions


In [50]:
set([str(e['exception']) for e in exceptions])

{'"could not find column column \'repository\'"'}

In [51]:
# let's do some analysis

In [54]:
path_template = '/home/leonhard/projects/tuplex-public/tuplex/cmake-build-debug/dist/bin/tests/BasicInvocationGithubProcessing/except_buf_{}.txt'

In [55]:
non_normal_data = []
for i in range(4):
    with open(path_template.format(i), 'r') as fp:
        non_normal_data += fp.readlines()

In [57]:
len(non_normal_data) # shiuld be 172391 

172391

In [58]:
# process all of these rows through the pipeline, how many exceptions will there be?

In [59]:
exceptions = []

num_normal_rows = 0
with open(output_path, 'w') as op:
    for row_no, line in tqdm(enumerate(non_normal_data)):
        res = pipeline_stage_0([line])
        if 'exception' in res.keys():
            exceptions.append(res)
        else:
        #process_row(row_no, line, op)
            num_normal_rows += 1

print('processed in total {} normal rows, {} exceptions'.format(num_normal_rows, len(exceptions)))

172391it [00:10, 15739.53it/s]

processed in total 161680 normal rows, 10711 exceptions


In [60]:
# so basically when normal-case concluded, there hsould be 
10725 - 10711

14

In [61]:
#exceptions

In [63]:
# check for 2013 which rows do NOT have respository key.
input_path = '/home/leonhard/Downloads/2013-10-15.json'
num_without_repository = 0
with open(input_path, 'r') as fp:
        for row_no, line in tqdm(enumerate(fp.readlines())):
            row = json.loads(line)
            if 'repository' not in row.keys():
                num_without_repository += 1
                print(row_no, row)
                
print(num_without_repository)

15174it [00:00, 76295.83it/s]

1 {'created_at': '2013-10-15T00:02:19-07:00', 'payload': {'id': 6987642, 'url': 'https://gist.github.com/6987642', 'name': 'gist: 6987642', 'desc': 'Bootstrap Customizer Config', 'action': 'create'}, 'public': True, 'type': 'GistEvent', 'url': 'https://gist.github.com/6987642', 'actor': None}
34 {'created_at': '2013-10-15T00:02:26-07:00', 'payload': {'id': 6987643, 'url': 'https://gist.github.com/6987643', 'name': 'gist: 6987643', 'desc': 'Bootstrap Customizer Config', 'action': 'create'}, 'public': True, 'type': 'GistEvent', 'url': 'https://gist.github.com/6987643', 'actor': None}
47 {'created_at': '2013-10-15T00:02:33-07:00', 'payload': {'target': {'id': 5641786, 'login': 'kaimeleon', 'followers': 1, 'repos': 0, 'gravatar_id': 'e573ef597467b54b76520b96034b7f20'}}, 'public': True, 'type': 'FollowEvent', 'url': 'https://github.com/kaimeleon', 'actor': 'hloehrmann', 'actor_attributes': {'login': 'hloehrmann', 'type': 'User', 'gravatar_id': '666ccde49edea882cf434c08db918a02', 'name': 'Ha

30480it [00:00, 75767.12it/s]

18336 {'created_at': '2013-10-15T01:38:53-07:00', 'payload': {'target': {'id': 629429, 'login': 'hakimel', 'followers': 3326, 'repos': 23, 'gravatar_id': '669d1a520df32153f9793e33c3905476'}}, 'public': True, 'type': 'FollowEvent', 'url': 'https://github.com/hakimel', 'actor': 'albertleao', 'actor_attributes': {'login': 'albertleao', 'type': 'User', 'gravatar_id': '0f4b7098083b52f3df49bafec1350c55', 'email': 'da39a3ee5e6b4b0d3255bfef95601890afd80709'}}
18359 {'created_at': '2013-10-15T01:39:02-07:00', 'payload': {'target': {'id': 64839, 'login': 'otfrom', 'followers': 62, 'repos': 78, 'gravatar_id': '324c1cedf36b59d44a13e76868af4b8d'}}, 'public': True, 'type': 'FollowEvent', 'url': 'https://github.com/otfrom', 'actor': 'locatefung', 'actor_attributes': {'login': 'locatefung', 'type': 'User', 'gravatar_id': '0a38a8400b89631d0225aade8eff67f4', 'email': 'da39a3ee5e6b4b0d3255bfef95601890afd80709'}}
18364 {'created_at': '2013-10-15T01:39:01-07:00', 'payload': {'id': 6988510, 'url': 'https://

45353it [00:00, 72028.60it/s]

...

269546 {'created_at': '2013-10-15T20:51:58-07:00', 'payload': {'target': {'id': 55072, 'login': 'ccpost', 'followers': 1, 'repos': 0, 'gravatar_id': 'ae532a5fc8ef6e3b16c1ddb9de88de18'}}, 'public': True, 'type': 'FollowEvent', 'url': 'https://github.com/ccpost', 'actor': 'Rhinon', 'actor_attributes': {'login': 'Rhinon', 'type': 'User', 'gravatar_id': '51a4b991e006ec0a2b81fdf54a943836', 'name': 'Ryan', 'location': 'United States', 'email': 'da39a3ee5e6b4b0d3255bfef95601890afd80709'}}
269598 {'created_at': '2013-10-15T20:52:12-07:00', 'payload': {'target': {'id': 189575, 'login': 'dzog', 'followers': 4, 'repos': 1, 'gravatar_id': '38ad9889827bc7d07f37c89a1c7c8e59'}}, 'public': True, 'type': 'FollowEvent', 'url': 'https://github.com/dzog', 'actor': 'Rhinon', 'actor_attributes': {'login': 'Rhinon', 'type': 'User', 'gravatar_id': '51a4b991e006ec0a2b81fdf54a943836', 'name': 'Ryan', 'location': 'United States', 'email': 'da39a3ee5e6b4b0d3255bfef95601890afd80709'}}
269600 {'created_at': '2013-1

291001it [00:04, 64390.63it/s]

283079 {'created_at': '2013-10-15T22:32:51-07:00', 'payload': {'id': 7003065, 'url': 'https://gist.github.com/7003065', 'name': 'gist: 7003065', 'desc': 'Rod Cutting Problem', 'action': 'create'}, 'public': True, 'type': 'GistEvent', 'url': 'https://gist.github.com/7003065', 'actor': 'jimmitt', 'actor_attributes': {'login': 'jimmitt', 'type': 'User', 'gravatar_id': '31235a8b902c415dbd395a9cbd073455', 'email': 'da39a3ee5e6b4b0d3255bfef95601890afd80709'}}
283153 {'created_at': '2013-10-15T22:33:32-07:00', 'payload': {'target': {'id': 728526, 'login': 'pfista', 'followers': 5, 'repos': 14, 'gravatar_id': '0849f52d59d90e91bee64287969df251'}}, 'public': True, 'type': 'FollowEvent', 'url': 'https://github.com/pfista', 'actor': 'ESGR', 'actor_attributes': {'login': 'ESGR', 'type': 'User', 'gravatar_id': 'c5b4cdb8d786fbbab5b38f856a9f5ad6', 'name': 'Evgeniy', 'email': '85e7034aaec9fad17c5ee4b57a4eae7f5077fc27@me.com'}}
283202 {'created_at': '2013-10-15T22:34:02-07:00', 'payload': {'id': 7003069

296456it [00:04, 64815.86it/s]

10725


In [ ]:
((str,
  Struct[(str,'shas'->List[(str,str,str,str,boolean)]),(str,'size'->i64),(str,'ref'->str),(str,'head'->str)],
  boolean,
  str,
  str,
  str,
  Struct[(str,'login'->str),(str,'type'->str),(str,'gravatar_id'->str),(str,'email'->str)],
  Struct[(str,'id'->i64),(str,'name'->str),(str,'url'->str),(str,'description'->str),(str,'watchers'->i64),(str,'stargazers'->i64),(str,'forks'->i64),(str,'fork'->boolean),(str,'size'->i64),(str,'owner'->str),(str,'private'->boolean),(str,'open_issues'->i64),(str,'has_issues'->boolean),(str,'has_downloads'->boolean),(str,'has_wiki'->boolean),(str,'language'->str),(str,'created_at'->str),(str,'pushed_at'->str),(str,'master_branch'->str)],i64)) -> i64